In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime as dt
from scipy import stats
import openpyxl
import calendar

In [2]:
# find difference between 2 dates in months

def month_diff(x,y):
    end = x.dt.to_period('M').view(dtype='int64')
    start = y.dt.to_period('M').view(dtype='int64')
    return end - start

In [3]:
# read transfer order

df = pd.read_excel('Transfer Order by Product5_31_2022 5 54 35 PM.xlsx', header=4)
df.columns = [i.lower() for i in df.columns]
df.columns = df.columns.str.replace(' ','_')
df.rename(columns={'value_in_base_currency':'sales','sales_person_name':'sales_rep',
                   'sales_order_date':'date'}, inplace=True)
df.date = pd.to_datetime(df.date, format='%d-%m-%Y')
df.account= [' '.join(i.split()) for i in df.account]
df.product_code=[' '.join(i.split()) for i in df.product_code]

df.rename(columns={'class':'cltv'},inplace=True)

df = df[['date','account_code','account','product_code','product_name','sales_rep','sales','cltv','country']]

## clustering address in philippines account list

In [4]:
# read account list

ac = pd.read_excel('sea_account.xlsx')
ac.columns = ac.columns.str.lower()
ac.columns = ac.columns.str.replace(' ','_')
ac.rename(columns={'class':'cltv','sales_executive':'sales_rep','region':'country'}, inplace=True)

In [5]:
# ac.postal_code= ac.postal_code.replace(' ',np.nan, regex=True)

In [6]:
ac1=ac[['account_code','account_name','country', 'city','postal_code','billing_address','cltv']]
ac1.country = [i.lower() for i in ac1.country]

In [7]:
ac1.head(2)

,account_code,account_name,country,city,postal_code,billing_address,cltv
0,SEA-002196,Deni Argo Clinic,indonesia,NaN,NaN,NaN,C
1,SEA-019560,Lee Dental HUB,philippines,NaN,NaN,NaN,C


In [8]:
ac1.country.unique()

array(['indonesia', 'philippines', 'malaysia', 'singapore', 'thailand',
       'vietnam'], dtype=object)

In [9]:
sg = ac1[ac1.country=='malaysia']

In [11]:
sg.billing_address = sg.billing_address.astype(str)

In [12]:
# extract postal code from billing address

import re

for i, n in zip(sg.index, sg.billing_address):
    
    # initializing K 
    K = 5
    # using regex() to solve problem
    temp = re.search('\d{% s}'% K, n)
    res = (temp.group(0) if temp else '')
    sg.loc[i,'postal_code']=res


In [13]:
def classify(cty):
    # compute number of days from last purchase
    _ = df[df.country==cty]
    _['current_date']=pd.to_datetime('today')
    _['since_last']=month_diff(_.current_date,_.date)
#     _=_[_.since_last >= 6]
    
    req = (_.since_last >= 6) & (_.since_last <= 24)
    _=_[req]
    class_a = _[_.cltv=='A']
    class_b = _[_.cltv=='B']
    class_c = _[_.cltv=='C']
    return class_a, class_b, class_c


## Ph account list to categorise accounts into clusters

In [14]:
sg.columns

Index(['account_code', 'account_name', 'country', 'city', 'postal_code',
       'billing_address', 'cltv'],
      dtype='object')

In [15]:
sg.postal_code = sg.postal_code.astype(str)

In [16]:
cty='Malaysia'
a,b,c=classify(cty)

# ab = a.append(b)

ab=pd.concat([a,b,c])


## Find out all the weekdays of the month

In [19]:
# yr=2022
def get_weekdays(mth,yr=2022):
    cal = calendar.monthcalendar(yr,mth)
    tot=[]
    for n in range(len(cal)):
        for i in range(0,5):
            a = cal[n][i]
            if a!=0:
                tot.append(a)
            else:
                continue
    return tot

In [20]:
yr=2022
mth=6
calendar.monthcalendar(yr,mth)

[[0, 0, 1, 2, 3, 4, 5],
 [6, 7, 8, 9, 10, 11, 12],
 [13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26],
 [27, 28, 29, 30, 0, 0, 0]]

In [21]:
mth=6
wkdays = get_weekdays(mth, yr)

In [22]:

class crm():
    def __init__(self,name, meeting, month, date, year, stime, etime,dayname, opp, desc, cocaller, cl, sales_rep):
        self.name = name
        self.meeting = meeting
        self.month = month
        self.date = date
        self.year = year
        self.stime = stime
        self.etime = etime
        self.dayname = dayname
        self.opp = opp
        self.desc = desc
        self.cocaller = cocaller
        self.cl=cl
        self.sales_rep = sales_rep
    
        
    def fdate(self):
        return "{}/{}/{}".format(self.month,self.date,  self.year)

In [23]:
t1="10:00"
t2="11:30"
t3='13:00'
t4="14:30"
t5="16:00"
t6="17:30"
t7="19:00"

timelist=[t1,t2,t3,t4,t5,t6,t7]

In [24]:
# groupby account code fo transfer order

a1 = ab.groupby(['account_code','account','sales_rep']).agg(no_pdts=('product_name','count')).reset_index()
a1.no_pdts = [str(i)+" "+ "products" for i in a1.no_pdts]

In [25]:
# attached cluster number to transfer order

a1['postal_code']=np.nan
for i,a in zip(a1.index,a1.account_code):
    for aa, c in zip(sg.account_code, sg.postal_code):
        if a==aa:
            a1.loc[i,'postal_code']=c

In [27]:
# create newly improved function

def plan1(tt,rep, mth,cl, s=0):
    
    wkdays = get_weekdays(mth,yr)
    
    rep = a1[a1.sales_rep==rep]
    m = rep[rep.postal_code==cl].reset_index(drop=True) # no of customers for that postal code
    if m.shape[0]<tt: 
        tt=m.shape[0]
    k = m.shape[0]//tt 
    
    account=[]
    meeting=[]
    date=[]
    start=[]
    end=[]
    dayname=[]
    oppy_pdt=[]
    description=[]
    cocaller=[]
    city=[]
    rep=[]
    e = s+k

    for n in wkdays[s:e]:
        for i in range(tt): #---------------------------------------------------------------
#             c1=crm(m.loc[i,'account'], 'telemarketing', mth,n,2022,timelist[i],timelist[i+1],'Y', m.loc[i,'no_pdts'],'N',cl, 
#                    m.loc[i,'sales_rep'])
            c1=crm(m.loc[i,'account'], 'telemarketing', mth,n,2022,timelist[i],timelist[i+1],calendar.weekday(2022,mth,n),'Y', m.loc[i,'no_pdts'],'N',cl, 
                   m.loc[i,'sales_rep'])
            account.append(c1.name)
            meeting.append(c1.meeting)
            date.append(c1.fdate())
            start.append(c1.stime)
            end.append(c1.etime)
            dayname.append(c1.dayname)
            oppy_pdt.append(c1.opp)
            description.append(c1.desc)
            cocaller.append(c1.cocaller)
            city.append(c1.cl)
            rep.append(c1.sales_rep)
            
            if i==tt-1: #------------------------------------------------------------------------
                m=m.loc[tt::].reset_index(drop=True) #--------------------------------------------
            else:
                continue
            
            
    df=pd.DataFrame({'account':account,'meeting':meeting,'date':date,'start':start,'end':end,'day':dayname,'oppy_pdt?':oppy_pdt,
               'description':description,'Co-caller?':cocaller,'city':cl,'sales_rep':rep})
    
    return df, e


def cluster(rep):
    _ = a1[a1.sales_rep==rep]
    return _.postal_code.value_counts().index


def index2int(dff,x):
    i = dff[dff.account==x].index.tolist()
    return int((" ".join(map(str, i))))


def index2str(i):
    return str((" ".join(map(str, i))))

def firstrow(p):
    _=p.iloc[[0]].T
    return _.T

## test with one district

In [28]:
# s=0
# tt=6
# rep='Audrey Ang'
# mth=6
# cl='14000'
# df, s = plan1(tt,rep,mth,cl,s)
# df


## Iterate over days of the month

In [29]:
# s=0
# mth=6
# rep='Audrey Ang'
# tt=6
# tot=pd.DataFrame()
# tot1=[]
# for cl in cluster(rep):
#     tot1.append(cl)
#     df, s = plan1(tt, rep,mth,cl,s)
#     tot=tot.append(df)

## Preference Allocator

In [30]:
# enter account preference details

pref = pd.DataFrame({'account':['Chew Dental Sergery','Goh Dental Surgery','Beam Dental'], 
                     'pref_time': [t1,t4,t1],'pref_day':[3,0,3],'city':['14000','08000','10350']})

# pref = pd.DataFrame({'account':['Tooth Matters Dental Surgery (Toa Payoh)'], 
#                      'pref_time': [t3],'pref_day':[2],'city':['d12']}, index=[0])

pref = pref.set_index('account')

## Account Preference Function

In [31]:
def crmplanner(pref, df):

    tot5=df.set_index('account')

    # extract the dataframe with selected city district from pref dataframe
    tot2=pd.DataFrame()
    for c in df.city.unique():
        for ct in pref.city.unique():
            if c==ct:
                aa = df[df.city==ct] 
                tot2=tot2.append(aa)

            else:
                continue

    aa = tot2.copy()
    


    # returns a list of accounts from selected district dataframe which tally with accounts with preferred time slot 
    tot3=[]
    for acc in pref.index.unique():
        for ac in aa.account.unique():
            if acc==ac:
                tot3.append(acc) 
                len_list = len(tot3)

            else:
                continue
                


    for i in tot3:
        # extract account and preferred T/S (df) from pref dataframe
        a_pref = pref.loc[[i]].T
        a_pref = a_pref.T

        # extract name of account from pref dataframe as a string
        a_pref_name = index2str(a_pref.index) # a_pref.index is based on iterator 'i'
        a_pref_city = a_pref.city[0]
        a_pref_time = a_pref.pref_time[0]
        a_pref_day = a_pref.pref_day[0]

        # tot5 is original dataframe (from df)
        # check to see whether a_pref_name, a_pref_city, a_pref_time are satisfied in a single customer
        # extract customer, city and start time from original list

        for s in tot5.index: # iterate over account names in original list
            if a_pref_name == s: # check whether pref account name is found in the original list
                a_orig = tot5.loc[[i]].T
                a_orig = a_orig.T
                a_orig_name = index2str(a_orig.index)
                a_orig_city = a_orig.city[0]
                a_orig_time = a_orig.start[0]
                a_orig_day = a_orig.day[0]



            else:

                continue

                # find out whether all conditions are satisfied for account name, city and preferred time
                # conditions satisfied
            if a_pref_name==a_orig_name and a_pref_city==a_orig_city and a_pref_time==a_orig_time and a_pref_day==a_orig_day:
                print(f"{a_pref_name}: Preferred day and time satisfied")
                print()
                continue # exit if all conditions are satisfied

            # conditions satisfied
            elif a_pref_name==a_orig_name and a_pref_city==a_orig_city and a_pref_time==a_orig_time:
                print(f"{a_pref_name}: Preferred time satisfied")
                print()
                continue # exit if all conditions are satisfied

            elif a_pref_name==a_orig_name and a_pref_city==a_orig_city:


                # extract original df with a_pref_city
                a_orig_df_city = tot5[tot5.city==a_orig_city]

                # check whether preferred day is in original crm list with selected district
                if a_pref_day in a_orig_df_city.day.tolist():


                    # extract df based on preferred day, preferred day available
                    a_pref_day_df = a_orig_df_city[a_orig_df_city.day==a_pref_day]

#                     num_of_appoints = a_orig_df_city[a_orig_df_city.day==a_pref_day]


                    # iterate over account name and start time to find a match with preferred time
                    for i, d in zip(a_pref_day_df.index,a_pref_day_df.start):

                        if d==a_pref_time: # preferred day available
                            
                            # extract preferred time
                            target_acct_with_pref_time = a_pref_day_df.loc[[i]].T
                            target_acct_with_pref_time = target_acct_with_pref_time.T

                            # extract name of target acct with preferred time
                            target_acct_with_pref_time_name = index2str(target_acct_with_pref_time.index)

                            # switch account names between 'target_acct_with_pref_time_name' and 'a_pref_name'
                            b_target_acct_with_pref_time=target_acct_with_pref_time.rename\
                            (index={target_acct_with_pref_time_name:a_pref_name})
                            b_orig = a_orig.rename(index={a_pref_name:target_acct_with_pref_time_name})

                            # delete these 2 names from original crm list "tot5"
                            tot5 = tot5.drop([target_acct_with_pref_time_name,a_pref_name])

                            tot5=tot5.append(b_target_acct_with_pref_time)
                            tot5=tot5.append(b_orig)

                            print(f"{a_pref_name}: Preferred day and time satisfied")
                            print()
                        else:
                            a_orig.loc[a_pref_name,'start']=a_pref_time
                            tot5=tot5.drop(a_pref_name)
                            tot5=tot5.append(a_orig)


                    # when the preferred time is in the original list, just switch place with another account        
                elif a_pref_time in a_orig_df_city.start.tolist():

                    target_acct_with_pref_time = a_orig_df_city[a_orig_df_city.start==a_pref_time]
                    target_acct_with_pref_time = firstrow(target_acct_with_pref_time)

                    # extract name of target acct with preferred time
                    target_acct_with_pref_time_name = index2str(target_acct_with_pref_time.index)

                    # switch account names between 'target_acct_with_pref_time_name' and 'a_pref_name'
                    b_target_acct_with_pref_time=target_acct_with_pref_time.rename\
                    (index={target_acct_with_pref_time_name:a_pref_name})
                    b_orig = a_orig.rename(index={a_pref_name:target_acct_with_pref_time_name})

                    # delete these 2 names from original crm list "tot5"
                    tot5 = tot5.drop([target_acct_with_pref_time_name,a_pref_name])

                    tot5=tot5.append(b_target_acct_with_pref_time)
                    tot5=tot5.append(b_orig)

                    print(f"{a_pref_name}: Preferred time satisfied")
                    print()


                    # when preferred time is not in the original list, just insert the time
                else:
                    a_orig.loc[a_pref_name,'start']=a_pref_time
                    tot5=tot5.drop(a_pref_name)
                    tot5=tot5.append(a_orig)
                    
    return tot5


In [32]:
# run auto planner function

s=0
mth=6
rep='Audrey Ang'
tt=6
tot=pd.DataFrame()
tot1=[]
for cl in cluster(rep):
    tot1.append(cl)
    df, s = plan1(tt, rep,mth,cl,s)
    tot=tot.append(df)

In [33]:
tot

,account,meeting,date,start,end,day,oppy_pdt?,description,Co-caller?,city,sales_rep
0,Dedentist Dental Surgery Sdn Bhd,telemarketing,6/1/2022,10:00,11:30,2.0,Y,2 products,N,14000,Audrey Ang
1,Klinik Pergigian Mesra (Bukit Mertajam),telemarketing,6/1/2022,11:30,13:00,2.0,Y,2 products,N,14000,Audrey Ang
2,B.T Tan Dental Surgery,telemarketing,6/1/2022,13:00,14:30,2.0,Y,7 products,N,14000,Audrey Ang
3,B.T.Tan Dental Clinic,telemarketing,6/1/2022,14:30,16:00,2.0,Y,1 products,N,14000,Audrey Ang
4,Chew Dental Sergery,telemarketing,6/1/2022,16:00,17:30,2.0,Y,38 products,N,14000,Audrey Ang
5,Dr Dentist Dental Surgery,telemarketing,6/1/2022,17:30,19:00,2.0,Y,2 products,N,14000,Audrey Ang
6,Ever White Dental Surgery,telemarketing,6/2/2022,10:00,11:30,3.0,Y,1 products,N,14000,Audrey Ang
7,Everwhite Dental Surgery,telemarketing,6/2/2022,11:30,13:00,3.0,Y,2 products,N,14000,Audrey Ang
8,Khor Dental Surgery,telemarketing,6/2/2022,13:00,14:30,3.0,Y,9 products,N,14000,Audrey Ang
9,Klinik Gigi Sentosa BM Sdn Bhd,telemarketing,6/2/2022,14:30,16:00,3.0,Y,3 products,N,14000,Audrey Ang


In [36]:
pref

,pref_time,pref_day,city
account,,,
Chew Dental Sergery,10:00,3,14000
Goh Dental Surgery,14:30,0,08000
Beam Dental,10:00,3,10350


In [34]:
pref=pref
df=tot
temp=crmplanner(pref,df)
temp=temp.reset_index()

Chew Dental Sergery: Preferred day and time satisfied

Goh Dental Surgery: Preferred day and time satisfied

Beam Dental: Preferred time satisfied



In [37]:
temp

,account,meeting,date,start,end,day,oppy_pdt?,description,Co-caller?,city,sales_rep
0,Dedentist Dental Surgery Sdn Bhd,telemarketing,6/1/2022,10:00,11:30,2.0,Y,2 products,N,14000,Audrey Ang
1,Klinik Pergigian Mesra (Bukit Mertajam),telemarketing,6/1/2022,11:30,13:00,2.0,Y,2 products,N,14000,Audrey Ang
2,B.T Tan Dental Surgery,telemarketing,6/1/2022,13:00,14:30,2.0,Y,7 products,N,14000,Audrey Ang
3,B.T.Tan Dental Clinic,telemarketing,6/1/2022,14:30,16:00,2.0,Y,1 products,N,14000,Audrey Ang
4,Dr Dentist Dental Surgery,telemarketing,6/1/2022,17:30,19:00,2.0,Y,2 products,N,14000,Audrey Ang
5,Everwhite Dental Surgery,telemarketing,6/2/2022,11:30,13:00,3.0,Y,2 products,N,14000,Audrey Ang
6,Khor Dental Surgery,telemarketing,6/2/2022,13:00,14:30,3.0,Y,9 products,N,14000,Audrey Ang
7,Klinik Gigi Sentosa BM Sdn Bhd,telemarketing,6/2/2022,14:30,16:00,3.0,Y,3 products,N,14000,Audrey Ang
8,Klinik Pergigian Kubang Semang,telemarketing,6/2/2022,16:00,17:30,3.0,Y,5 products,N,14000,Audrey Ang
9,LH Chong Dental Surgery,telemarketing,6/2/2022,17:30,19:00,3.0,Y,10 products,N,14000,Audrey Ang
